<a href="https://colab.research.google.com/github/mina0511/AICP_URECORR/blob/main/AICP_RSS_STUDY_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser goose3 beautifulsoup4 tqdm

In [ ]:
import pandas as pd #pandas 객체 처리
import feedparser 
import requests #웹 소켓 통신
from goose3 import Goose 
from goose3.text import StopWordsKorean
from bs4 import BeautifulSoup #마크업 언어 파싱
from tqdm.notebook import tqdm #상태 바 표시

In [ ]:
rss_df = pd.read_pickle('rss_list.pkl')
print(rss_df.head())

    corp          name                                                url
34  조선닷컴  [뉴스]오늘의 주요뉴스   http://myhome.chosun.com/rss/www_section_rss.xml
35  조선닷컴        [뉴스]정치   http://www.chosun.com/site/data/rss/politics.xml
36  조선닷컴        [뉴스]국제  http://www.chosun.com/site/data/rss/internatio...
37  조선닷컴        [뉴스]문화    http://www.chosun.com/site/data/rss/culture.xml
38  조선닷컴     [뉴스]사설ㆍ칼럼  http://www.chosun.com/site/data/rss/editorials...


In [ ]:
len(rss_df)

188

In [ ]:
sources = rss_df.corp.unique() # unique 함수: 주어진NumPy 배열의 모든 고유 값을 검색하고 이러한 고유 값을 정렬
sources

array(['조선닷컴', '노컷뉴스', '동아닷컴', '세계일보', '매일경제', '경향닷컴', '한국아이닷컴', '파이낸셜뉴스',
       '헤럴드경제', '데이터넷', '탑라이더', '뉴스포스트', '뉴스데일리', '에버뉴스', '주간포커스', '다음뉴스',
       '구글뉴스', '중부매일', '르몽드 디플로마티크'], dtype=object)

In [ ]:
def get_data(url):
    try:
        res = requests.get(url)
        html = res.text
        data = feedparser.parse(html)
        print(data.feed.title)
        return data
    except:
        return None

In [ ]:
import re #정규표현식 모듈
txt = '>2021' #target 텍스트
p = re.compile(txt) #정규식 컴파일
url_list = [] #url 목록이 저장될 리스트

for url in tqdm(rss_df['url']): #rss_df에서 url을 모두 추출
  try:
    if (p.search(requests.get(url).text)) != None: #정규식과 일치하는 문자가 있을 경우
      url_list.append(url) #url 리스트에 추가
  except:
    pass
print(url_list)


['http://www.chosun.com/site/data/rss/politics.xml', 'http://www.chosun.com/site/data/rss/international.xml', 'http://www.chosun.com/site/data/rss/culture.xml', 'http://www.chosun.com/site/data/rss/editorials.xml', 'http://biz.chosun.com/site/data/rss/rss.xml', 'http://biz.chosun.com/site/data/rss/news.xml', 'http://biz.chosun.com/site/data/rss/market.xml', 'http://biz.chosun.com/site/data/rss/estate.xml', 'http://www.chosun.com/site/data/rss/sports.xml', 'http://www.chosun.com/site/data/rss/ent.xml', 'http://health.chosun.com/site/data/rss/rss.xml', 'http://careview.chosun.com/site/data/rss/rss.xml', 'http://art.chosun.com/site/data/rss/rss.xml', 'http://health.chosun.com/rss/column.xml', 'http://rss.donga.com/economy.xml', 'http://rss.donga.com/sports.xml', 'http://rss.donga.com/sportsdonga/golf.xml', 'http://file.mk.co.kr/news/rss/rss_30000001.xml', 'http://file.mk.co.kr/news/rss/rss_30000023.xml', 'http://file.mk.co.kr/news/rss/rss_40300001.xml', 'http://file.mk.co.kr/news/rss/rss

In [ ]:
news_list = [] #뉴스 링크가 저장될 리스트

for url in tqdm(url_list): #url 리스트에서 모든 url을 가져옴
  try:  #에러가 일어날 경우 pass
    xml = requests.get(url).text.replace('link', 'aicp') #GET 요청으로 XML을 가져온 뒤, link 태그를 aicp로 변경
    soup =  BeautifulSoup(xml, 'html.parser') #BeautifulSoup 객체 생성
    item_list = soup.find_all('item') #item 태그를 포함한 모든 객체를 리스트 형태로 가져옴
    for item in item_list: #item_list 에서 각 item을 가져옴
      news_list.extend([news.get_text() for news in item.find_all('aicp')]) #item 태그 하위의 모든 aicp 태그를 추출해 aicp 태그 사이의 url 추출
  except:
    print(url, 'is not valid url')
    pass

In [ ]:
import re
txt = '삼성전자'
p = re.compile(txt)
samsung_url_list = []

for url in tqdm(news_list):
  try:
    if (p.search(requests.get(url).text)) != None: 
      samsung_url_list.append(url)
  except:
    pass
print(samsung_url_list)


['https://www.donga.com/news/It/article/all/20210323/106036024/1', 'https://www.donga.com/news/Economy/article/all/20210323/106029295/1', 'https://www.donga.com/news/Economy/article/all/20210323/106027916/1', 'https://www.donga.com/news/Society/article/all/20210323/106026262/1', 'http://view.koreaherald.com/kh/view.php?ud=20210323000756', 'http://view.koreaherald.com/kh/view.php?ud=20210323000722', 'http://view.koreaherald.com/kh/view.php?ud=20210323000710', 'http://view.koreaherald.com/kh/view.php?ud=20210323000703', 'http://view.koreaherald.com/kh/view.php?ud=20210323000694', 'http://view.koreaherald.com/kh/view.php?ud=20210323000528', 'http://view.koreaherald.com/kh/view.php?ud=20210323000516', 'http://view.koreaherald.com/kh/view.php?ud=20210316000593', 'http://www.datanet.co.kr/news/articleView.html?idxno=157523', 'http://www.datanet.co.kr/news/articleView.html?idxno=157482', 'http://www.newspost.kr/news/articleView.html?idxno=92616', 'http://www.newsdaily.kr/news/articleView.htm